# Wilson Cowan Tutorial

In this tutorial, we will be learning how to use NeuroTorch to train a recurrent neural network that reproduce time series using the Wilson-Cowan dynamic.

**Plan of this notebook**
1. Brief introduction to the theory behind the Wilson-Cowan dynamic
2. Setup
3. How to create Dataset and dataloader
4. Setting up our data
5. How to run the training
6. Tools to visualise the trained data

---

### Wilson-Cowan dynamic

Introduce in 1972 by Wilson HR and Cowan JD, the Wilson-Cowan dynamic performs well when it comes to reproducing and predicting calcium neuronal activity. The equation used is the following :

# <center> $\tau \dot{\vec{x}} = -\vec{x} + (1 - \vec{r} \odot \vec{x}) \odot \sigma({\mathbf{W}\vec{x} - \vec{\mu}})$

We can apply Euler's discretisation. By doing so, we obtain :

# <center> $\vec{x}_{(t+1)} = \vec{x}_{(t)}\left(1- \frac{\Delta t}{\tau}\right) + \frac{\Delta t}{\tau}\left(\left(1 - \vec{r}\odot\vec{x}_{(t)}\right)\odot\sigma\left(\mathbf{W}\vec{x} - \vec{\mu}\right)\right)$


However, in NeuroTorch, we slightly modify this equation to allow to use of batches and to be coherent with format that what use in the rest of the module. In neuroscience, it is common to represent the neuronal activity with a matrix ($N$ x $T$) where $N$ is the number of neurons and $T$ is the number of time step. However, in NeuroTorch, we use the following convention : ($B$ x $T$ x $N$) where $B$ is the number of batch. In this notebook, our goal is to reproduce a specific time series, therefore, our batch size will be $B = 1$. We therefore apply a transpose on our weight matrix and inputs. With this in mind, the Wilson-Cowan becomes :

# <center> $\vec{x}_{(t+1)} = \vec{x}_{(t)}\left(1 - \frac{\Delta t}{\tau}\right) + \frac{\Delta t}{\tau}\left(\left(1 - \vec{r}\odot\vec{x}_{(t)}\right)\odot \sigma\left(\vec{x}\mathbf{W} - \mu\right)\right)$

By doing so, we obtain the same result but we are now using an equation that follow what have been done so far in NeuroTorch.

Now, let us define the different parameters of this equation:

- $\vec{x}$ is the neuronal activity at a time $t$ (normalized between 0 and 1) of size (1 x $N$)
- $\Delta t$ is the time step of integration
- $\tau$ is the characteristic time. It represents the time needed for the neuronal activity to decay
- $\vec{r}$ is the refractory time. It represents the time needed for a neuron to fire again once she has already fired of size (1 x $N$)
- $\sigma$ is the sigmoid function
- $W$ is the weight connexion (connectome) of size ($N$ x $N$). $W_{ij}$ tell us how neuron $i$ is connected to neuron $j$
- $\mu$ is the activation threshold of size (1 x $N$). It represents the input needed for a neuron to fire. A neuron with a small $\mu$ will be sensitive to input and will easily fire.

---

### Setup

For those not familiar with Python and virtual environment, you can:

- Follow those steps if you understand french : [venv and pycharm](https://github.com/JeremieGince/TutorielPython-Manuel/tree/master/Environments)

- For english speaker : TODO

You can now install the dependencies by running the following commands :


In [ ]:
!pip install -r time_series_forcasting_wilson_cowan_requirements.txt
!pip install git+https://github.com/JeremieGince/NeuroTorch.git

If you have a cuda device and want to use it for this tutorial, you can uninstall pytorch with `pip uninstall torch` and re-install it with the right cuda version by generating a command with [PyTorch GetStarted](https://pytorch.org/get-started/locally/) web page. However, for this notebook, we recommend using CPU since we are not using batches.

After setting up the virtual environment, we will need to import the necessary packages.

These are the packages that will be needed for NeuroTorch:

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import neurotorch as nt
from neurotorch.metrics import RegressionMetrics
from neurotorch.visualisation.time_series_visualisation import Visualise, VisualiseKMeans
from neurotorch.regularization.connectome import DaleLawL2
from neurotorch import WilsonCowanLayer
from src.neurotorch.transforms.base import to_tensor

---
### Dataset & Dataloader

When working with PyTorch, it is important to define a dataset and a dataloader. Of course, you can always use the one that are provided in this module. However, you might be interested in knowing how those simple class are built. With NeuroTorch, multiple datasets and dataloader are already built-in depending on what you need. Dataset are used to prepare your datas to be used in NeuroTorch. For instance, dataset are used to inform NeuroTorch on the transformation you want to apply to you data or the ratio of datas that will be used to training and to evaluate the quality of the training.

1. First, you begin by creating the constructor
2. You redefine the <code>\_\_len\_\_</code> operator. It returns the size of the dataset
3. You redefine the <code>\_\_getitem__</code> operator. You want to be able to use indexing such as <code> dataset[i] </code> can be use to get the *i*th sample in your dataset. You return the value needed for training and testing.

In our case, we have a dataset of size 1 (one sample) since we only use one time series. Also, when defining <code>\_\_getitem__</code>, we will return the initial condition and the rest of the timeseries since we only have one dataset. We unsqueeze the initiale condition so the tensor goes from size [$N$] to size [1 x $N$]. Remember that, in this dataset, your time series must have the size [$T$ x $N$]

In [ ]:
class WCDataset(Dataset):
	def __init__(self, x):
		self.x = x

	def __len__(self):
		return 1

	def __getitem__(self, item):
		return torch.unsqueeze(self.x[0], dim=0), self.x[1:]

For the dataloader, we can simply use the dataloader built in PyTorch. Sometime, it is interesting to build our own. If this case applies to you, feel free look at a few dataloaders and datasets that were built for WilsonCowan in application/time_series_forecasting_wilson_cowan/dataset

---
### Setting up our data

This section depend heavily on the way you have your data. For this tutorial, we will use a neuronal activity from a zebra fish. We will take 200 random neurons in our dataset. In this dataset, 406 time steps are present. We begin by importing the data :



In [16]:
import numpy as np

time_series = np.load('timeSeries_2020_12_16_cr3_df.npy')
n_neurons, n_shape = time_series.shape
sample_size = 200
sample = np.random.randint(n_neurons, size=sample_size)
data = time_series[sample, :]


To improve the training, one might also want to smooth the data :

In [19]:
from scipy.ndimage import gaussian_filter1d

sigma = 20

for neuron in range(data.shape[0]):
	data[neuron, :] = gaussian_filter1d(data[neuron, :], sigma=sigma)
	data[neuron, :] = data[neuron, :] - np.min(data[neuron, :])
	data[neuron, :] = data[neuron, :] / np.max(data[neuron, :])



---
### Training the data

To reproduce a time series, you can follow those easy steps :

1. Think about the parameters of your training. What kind of optimizer, learning rate, regularisation ...
2. Define your function <code> train_with_params </code>
3. Run the code!

With NeuroTorch, you can easily use the optimizer, loss function, regularisation ... that you want! However, when it comes to reproducing time series, we have selected the one that performs the best. For this tutorial, we will use those.

#### Defining <code> train_with_params </code>

As you can see, until now, very few lines of codes were needed. <code> train_with_params </code> is the only function that must be definie by the user because this is where you tell NeuroTorch the parameters you wish to use. We will walk you through the process, so you can easily recreate this function or even modify to fit your needs. <code> train_with_params </code> takes 3 kind of entries :

- Training related arguments
- Wilson-Cowan dynamic arguments
- device to compute

Generally, you want your argument to be aspect of your training that might change from training to training. For instance, if you're planinng on always using the same learning rate, you might want to remove learning rate from <code> train_with_params </code> and directly add this parameters in the code rather than as an arguments. Of course, the opposite can also be done.

In [ ]:
from typing import *

def train_with_params(
		true_time_series: np.ndarray or torch.Tensor,
		learning_rate: float = 1e-2,
		epochs: int = 100,
		forward_weights: Optional[torch.tensor or np.ndarray] = None,
		std_weights: float = 1.0,
		dt: float = 0.02,
		mu: Optional[float or torch.Tensor or np.ndarray] = 0.0,
		mean_mu : Optional[float] = 0.0,
		std_mu: Optional[float] = 1.0,
		r: Optional[float or torch.Tensor or np.ndarray] = 1.0,
		mean_r: Optional[float] = 1.0,
		std_r: Optional[float] = 1.0,
		tau: float = 1.0,
		learn_mu: bool = False,
		learn_r: bool = False,
		learn_tau: bool = False,
		device: torch.device = torch.device("cpu")
):
	pass

From there, we can build this function by following these steps :

1. Initialise layer
2. Initialise sequential
3. Initialise trainer

##### Step 1.

In [ ]:
def train_with_params(
		true_time_series: np.ndarray or torch.Tensor,
		learning_rate: float = 1e-2,
		epochs: int = 100,
		forward_weights: Optional[torch.tensor or np.ndarray] = None,
		std_weights: float = 1.0,
		dt: float = 0.02,
		mu: Optional[float or torch.Tensor or np.ndarray] = 0.0,
		mean_mu : Optional[float] = 0.0,
		std_mu: Optional[float] = 1.0,
		r: Optional[float or torch.Tensor or np.ndarray] = 1.0,
		mean_r: Optional[float] = 1.0,
		std_r: Optional[float] = 1.0,
		tau: float = 1.0,
		learn_mu: bool = False,
		learn_r: bool = False,
		learn_tau: bool = False,
		device: torch.device = torch.device("cpu")
):
	if not torch.is_tensor(true_time_series):  # convert time series to torch tensor
		true_time_series = torch.tensor(true_time_series, dtype=torch.float32, device=device)
	x = true_time_series.T[np.newaxis, :]  # ensure that time series has format (1 x T x N)
	ws_layer = WilsonCowanLayer(
		x.shape[-1], x.shape[-1],
		forward_weights=forward_weights,
		std_weights=std_weights,
		dt=dt,
		r=r,
		mean_r=mean_r,
		std_r=std_r,
		mu=mu,
		mean_mu=mean_mu,
		std_mu=std_mu,
		tau=tau,
		learn_r=learn_r,
		learn_mu=learn_mu,
		learn_tau=learn_tau,
		device=device,
		name="WilsonCowan"
	)